In [1]:
import transformers
import torch
import pandas as pd

/home/bracke/miniconda3/envs/gpu-venv-transnormer/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set-up device, tokenizer, model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = transformers.T5ForConditionalGeneration.from_pretrained("/home/bracke/code/transnormer/models/model/checkpoint-800").to(device)
# tokenizer = transformers.AutoTokenizer.from_pretrained("google/byt5-small")
model = transformers.AutoModel.from_pretrained("/home/bracke/code/transnormer/models/models_2023-03-16_15-24/checkpoint-100000").to(device)

print(type(model))
print(type(tokenizer))

<class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>
<class 'transformers.models.byt5.tokenization_byt5.ByT5Tokenizer'>


In [20]:
# Input string
s_orig = "Wie ſie jhre Freunde ſo ſie beſuch en empfangen."
s_norm = 'Wie sie ihre Freunde so sie besuche en empfangen.'
# s_orig = "Nach etlichen Taͤntzen kam vnſer Præſidentin wider daher/ vnd vermeldet vns wie das ſich die Kuͤnſtler vnd Studioſi gegen jhrer Koͤnig. Majeſt. erbotten/ deren zu ehren/ vnd gefallen vor dero abzug/ ein froͤliche Comœdiam zu agieren/ wolten nuhn wir derſelben auch beywohnen/ vnd Koͤnig. Ma. auff der Sonnen Hauß begleiten/ daß were dero Lieb/ vnd wolte ſolches in allen gnaden erkennen:"
# s_norm = "Nach etlichen Tänzen kam unser Präsidentin wider daher/ und vermeldet uns wie das sich die Künstler und Studiosi gegen ihrer König. Majest. erboten/ deren zu ehren/ und gefallen vor deren Abzug/ ein fröhliche Comoediam zu agieren/ wollten nun wir derselben auch beiwohnen/ und König. Ma. auf der Sonnen Haus begleiten/ dass wäre deren Liebe/ und wollte solches in allen Gnaden erkennen:" 

s_orig = "ſolen " #* 10
s_norm = "Solen " #* 150

# s_norm = "foo bar" 
inputs = tokenizer(s_orig, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
outputs = tokenizer(s_norm, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
input_ids = inputs.input_ids.to(device)
output_ids = outputs.input_ids.to(device)
output_ids[output_ids==0] = -100 # replace padding token with -100
attention_mask = inputs.attention_mask.to(device)
example = {
    "input_ids":input_ids,
    "attention_mask":attention_mask,
    "labels":output_ids
}


In [21]:
# Loss
output_forward = model(**example)
output_forward.loss

tensor(0.2410, device='cuda:0', grad_fn=<NllLossBackward0>)

In [16]:
# Generation
output_gen = model.generate(
    input_ids, 
    attention_mask=attention_mask, 
    max_length=256,
    no_repeat_ngram_size = 3,
    early_stopping = True,
    length_penalty = 2.0,
    num_beams = 4,
    
    )
# del input_ids, attention_mask # free memory
output_str = tokenizer.batch_decode(output_gen, skip_special_tokens=True)[0]
print(output_str)

Solen 


In [6]:
# Display output
from IPython.core.display import HTML
df_example = pd.DataFrame(data=output_str)
display(HTML(df_example.head(1).to_html(escape=False)))

,0
0,Abc


---

# Sentences from the train set

'Nach etlichen Taͤntzen kam vnſer Præſidentin wider daher/ vnd vermeldet vns wie das ſich die Kuͤnſtler vnd Studioſi gegen jhrer Koͤnig. Majeſt. erbotten/ deren zu ehren/ vnd gefallen vor dero abzug/ ein froͤliche Comœdiam zu agieren/ wolten nuhn wir derſelben auch beywohnen/ vnd Koͤnig. Ma. auff der Sonnen Hauß begleiten/ daß were dero Lieb/ vnd wolte ſolches in allen gnaden erkennen:'

CAB-norm: 'Nach etlichen Tänzen kam unser Präsidentin wider daher/ und vermeldet uns wie das sich die Künstler und Studiosi gegen ihrer König. Majest. erboten/ deren zu ehren/ und gefallen vor deren Abzug/ ein fröhliche Comoediam zu agieren/ wollten nun wir derselben auch beiwohnen/ und König. Ma. auf der Sonnen Haus begleiten/ dass wäre deren Liebe/ und wollte solches in allen Gnaden erkennen:'